# Scapy

## basic

**Import scapy**

In [1]:
from scapy.all import *

**How to build packets**

In [2]:
IP()

<IP  |>

In [34]:
IP(dst="10.0.2.4")

<IP  dst=10.0.2.4 |>

**Packet composed over two layers (IP - layer 3, TCP - layer 4). / operator is used to compose over layers.**

In [2]:
p = IP(dst="10.0.2.4")/TCP()/"Hello world!"
p

<IP  frag=0 proto=tcp dst=10.0.2.4 |<TCP  |<Raw  load='Hello world!' |>>>

In [39]:
hexdump(p)

0000   45 00 00 34 00 01 00 00  40 06 62 BB 0A 00 02 05   E..4....@.b.....
0010   0A 00 02 04 00 14 00 50  00 00 00 00 00 00 00 00   .......P........
0020   50 02 20 00 05 7B 00 00  48 65 6C 6C 6F 20 77 6F   P. ..{..Hello wo
0030   72 6C 64 21                                        rld!


**Lets move for a while to the second layer:**

In [4]:
p = Ether()/IP(dst="10.0.2.4")/TCP()/"Hello world!"
p

<Ether  type=0x800 |<IP  frag=0 proto=tcp dst=10.0.2.4 |<TCP  |<Raw  load='Hello world!' |>>>>

In [5]:
ls(p)

dst        : DestMACField                        = '08:00:27:f8:a1:68' (None)
src        : SourceMACField                      = '08:00:27:59:1b:51' (None)
type       : XShortEnumField                     = 2048            (36864)
--
version    : BitField (4 bits)                   = 4               (4)
ihl        : BitField (4 bits)                   = None            (None)
tos        : XByteField                          = 0               (0)
len        : ShortField                          = None            (None)
id         : ShortField                          = 1               (1)
flags      : FlagsField (3 bits)                 = 0               (0)
frag       : BitField (13 bits)                  = 0               (0)
ttl        : ByteField                           = 64              (64)
proto      : ByteEnumField                       = 6               (0)
chksum     : XShortField                         = None            (None)
src        : SourceIPField (Emph)             

In [52]:
str(p)

'E\x00\x004\x00\x01\x00\x00@\x06b\xbb\n\x00\x02\x05\n\x00\x02\x04\x00\x14\x00P\x00\x00\x00\x00\x00\x00\x00\x00P\x02 \x00\x05{\x00\x00Hello world!'

In [6]:
p.show()

###[ Ethernet ]### 
  dst       = 08:00:27:f8:a1:68
  src       = 08:00:27:59:1b:51
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = None
     src       = 10.0.2.5
     dst       = 10.0.2.4
     \options   \
###[ TCP ]### 
        sport     = ftp_data
        dport     = http
        seq       = 0
        ack       = 0
        dataofs   = None
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = None
        urgptr    = 0
        options   = {}
###[ Raw ]### 
           load      = 'Hello world!'



**How to modify packets:**

In [8]:
p[IP].src

'10.0.2.5'

In [9]:
p[IP].src='10.0.2.99'

In [11]:
p.show()

###[ Ethernet ]### 
  dst       = 08:00:27:f8:a1:68
  src       = 08:00:27:59:1b:51
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = None
     src       = 10.0.2.99
     dst       = 10.0.2.4
     \options   \
###[ TCP ]### 
        sport     = ftp_data
        dport     = http
        seq       = 0
        ack       = 0
        dataofs   = None
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = None
        urgptr    = 0
        options   = {}
###[ Raw ]### 
           load      = 'Hello world!'



## sending and receiving packets - ICMP example

**Create IP packet with ICMP protocol**
* dst - is destination address, or addresses (please note that example uses range from 1 to 10)
* timeout - in second how long scapy should wait after last sent packet
* "Hello world" - packet payload

**Respone will be tuple with two elements:**
* answered packets
* unanswered packets

**To just sent packet use send command**

In [18]:
send(IP(dst="10.0.2.5")/ICMP()/"Hello world!")


Sent 1 packets.


T**o send and receive one received packet use sr1 method**

In [19]:
ans = sr1(IP(dst="10.0.2.4")/ICMP()/"Hello world!")

Begin emission:
Finished to send 1 packets.

Received 28 packets, got 1 answers, remaining 0 packets


In [21]:
ans.summary()

'IP / ICMP 10.0.2.4 > 10.0.2.5 echo-reply 0 / Raw / Padding'

In [22]:
ans.show()

###[ IP ]### 
  version   = 4L
  ihl       = 5L
  tos       = 0x0
  len       = 40
  id        = 36863
  flags     = 
  frag      = 0L
  ttl       = 64
  proto     = icmp
  chksum    = 0xd2cd
  src       = 10.0.2.4
  dst       = 10.0.2.5
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0x8e10
     id        = 0x0
     seq       = 0x0
###[ Raw ]### 
        load      = 'Hello world!'
###[ Padding ]### 
           load      = '\x00\x00\x00\x00\x00\x00'



**To send and receive all packets use sr method**

In [8]:
ans,unans=sr(IP(dst="10.0.2.1-10")/ICMP()/"Hello world!",timeout=2)

Begin emission:
Finished to send 10 packets.

Received 92 packets, got 3 answers, remaining 7 packets


In [9]:
ans.summary()

IP / ICMP 10.0.2.5 > 10.0.2.1 echo-request 0 / Raw ==> IP / ICMP 10.0.2.1 > 10.0.2.5 echo-reply 0 / Raw / Padding
IP / ICMP 10.0.2.5 > 10.0.2.2 echo-request 0 / Raw ==> IP / ICMP 10.0.2.2 > 10.0.2.5 echo-reply 0 / Raw / Padding
IP / ICMP 10.0.2.5 > 10.0.2.4 echo-request 0 / Raw ==> IP / ICMP 10.0.2.4 > 10.0.2.5 echo-reply 0 / Raw / Padding


**Let's make some changes in the request:**
* src - it's source address
* ttl - time to live 

In [14]:
ans,unans=sr(IP(src="10.0.2.2",dst="10.0.2.4",ttl=128)/ICMP()/"Hello world!",timeout=2)

Begin emission:
Finished to send 1 packets.

Received 49 packets, got 0 answers, remaining 1 packets


In [15]:
ans.summary()

**Source address has been changed to other machine thus we do not receive any response - guess who received ;)**

**Let's display all unanswered packets.**

In [16]:
unans.summary()

IP / ICMP 10.0.2.2 > 10.0.2.4 echo-request 0 / Raw


**Not enough, OK, we can also send ping reply in behalf of 10.0.2.4**

**ICMP type 0 means - echo reply, let's create such packet**

In [17]:
send(IP(src="10.0.2.4", dst="10.0.2.2", ttl=128)/ICMP(type=0)/"HelloWorld") 


Sent 1 packets.


## Simple port scanner

**Let's create basic TCP/IP packet**

In [23]:
ans,unans = sr(IP(dst="10.0.2.4")/TCP(dport=23))

Begin emission:
Finished to send 1 packets.

Received 26 packets, got 1 answers, remaining 0 packets


In [24]:
ans.summary()

IP / TCP 10.0.2.5:ftp_data > 10.0.2.4:telnet S ==> IP / TCP 10.0.2.4:telnet > 10.0.2.5:ftp_data SA / Padding


In [25]:
ans,unans = sr(IP(dst="10.0.2.4")/TCP(dport=678))

Begin emission:
Finished to send 1 packets.

Received 25 packets, got 1 answers, remaining 0 packets


In [26]:
ans.summary()

IP / TCP 10.0.2.5:ftp_data > 10.0.2.4:678 S ==> IP / TCP 10.0.2.4:678 > 10.0.2.5:ftp_data RA / Padding


**Please note response flags:**
* port 23 - response is SA which means SYN-ACK (port open)
* port 678 - response is RA which means RESET-ACK (port closed)

**Would you like to have more control over sent packet, no problem:**
* sport - source port
* dport - list of ports instead of one port!
* flag - request flag (here S - SYN)

In [28]:
ans,unans = sr(IP(dst="10.0.2.4")/TCP(sport=777,dport=[23,80,10000],flags="S"))

Begin emission:
Finished to send 3 packets.

Received 28 packets, got 3 answers, remaining 0 packets


In [29]:
ans.summary()

IP / TCP 10.0.2.5:moira_update > 10.0.2.4:telnet S ==> IP / TCP 10.0.2.4:telnet > 10.0.2.5:moira_update SA / Padding
IP / TCP 10.0.2.5:moira_update > 10.0.2.4:http S ==> IP / TCP 10.0.2.4:http > 10.0.2.5:moira_update SA / Padding
IP / TCP 10.0.2.5:moira_update > 10.0.2.4:webmin S ==> IP / TCP 10.0.2.4:webmin > 10.0.2.5:moira_update RA / Padding


**OK, maybe is worth to add sport randomized + some retries?**
* sport - RandShort(), random source port
* flags - S = Syn
* inter - time interval between two packets,
* retry - number of retries
* timeout - how long scapy should wait after last sent packet

In [31]:
ans,unans = sr(IP(dst="10.0.2.4")/TCP(sport=RandShort(),dport=80,flags="S"),inter=0.5,retry=2,timeout=1)

Begin emission:
Finished to send 1 packets.

Received 27 packets, got 1 answers, remaining 0 packets


In [32]:
ans.summary()

IP / TCP 10.0.2.5:23174 > 10.0.2.4:http S ==> IP / TCP 10.0.2.4:http > 10.0.2.5:23174 SA / Padding


## ARP ping

In [53]:
arping("10.0.2.*")

Begin emission:
Finished to send 256 packets.

Received 4 packets, got 4 answers, remaining 252 packets
  52:54:00:12:35:00 10.0.2.1
  08:00:27:67:15:c9 10.0.2.3
  52:54:00:12:35:00 10.0.2.2
  08:00:27:f8:a1:68 10.0.2.4


(<ARPing: TCP:0 UDP:0 ICMP:0 Other:4>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:252>)

## TCP ping

In [2]:
ans,unans=sr(IP(dst="10.0.2.0-10")/TCP(dport=80, flags="S"),timeout=4)

Begin emission:
Finished to send 11 packets.

Received 91 packets, got 3 answers, remaining 8 packets


In [4]:
ans.summary()

IP / TCP 10.0.2.5:ftp_data > 10.0.2.1:http S ==> IP / TCP 10.0.2.1:http > 10.0.2.5:ftp_data RA / Padding
IP / TCP 10.0.2.5:ftp_data > 10.0.2.2:http S ==> IP / TCP 10.0.2.2:http > 10.0.2.5:ftp_data SA / Padding
IP / TCP 10.0.2.5:ftp_data > 10.0.2.4:http S ==> IP / TCP 10.0.2.4:http > 10.0.2.5:ftp_data SA / Padding


## UDP ping

In [2]:
ans,unans=sr(IP(dst="10.0.2.0-10")/UDP(dport=1))

Begin emission:
Finished to send 11 packets.

Received 137 packets, got 2 answers, remaining 9 packets


In [3]:
ans.summary()

IP / UDP 10.0.2.5:domain > 10.0.2.1:1 ==> IP / ICMP 10.0.2.1 > 10.0.2.5 dest-unreach port-unreachable / IPerror / UDPerror
IP / UDP 10.0.2.5:domain > 10.0.2.4:1 ==> IP / ICMP 10.0.2.4 > 10.0.2.5 dest-unreach port-unreachable / IPerror / UDPerror


## Traceroute

In [ ]:
traceroute(["www.google.com"], maxttl=20)

## DNS query

**First of all check what can be set for DNS question record**

In [4]:
DNSQR().show()

###[ DNS Question Record ]### 
  qname     = 'www.example.com'
  qtype     = A
  qclass    = IN



In [3]:
DNS().show()

###[ DNS ]### 
  id        = 0
  qr        = 0
  opcode    = QUERY
  aa        = 0
  tc        = 0
  rd        = 1
  ra        = 0
  z         = 0
  ad        = 0
  cd        = 0
  rcode     = ok
  qdcount   = 0
  ancount   = 0
  nscount   = 0
  arcount   = 0
  qd        = None
  an        = None
  ns        = None
  ar        = None



**DNS is UDP packet, so create IP()/UDP()/DNS() packet**

In [2]:
ans = sr1(IP(dst="8.8.8.8")/UDP()/DNS(rd=1,qd=DNSQR(qname="www.google.com")),timeout=5)

Begin emission:
Finished to send 1 packets.

Received 46 packets, got 1 answers, remaining 0 packets


In [5]:
ans.show()

###[ IP ]### 
  version   = 4L
  ihl       = 5L
  tos       = 0x0
  len       = 76
  id        = 65167
  flags     = 
  frag      = 0L
  ttl       = 118
  proto     = udp
  chksum    = 0x73f1
  src       = 8.8.8.8
  dst       = 192.168.1.104
  \options   \
###[ UDP ]### 
     sport     = domain
     dport     = domain
     len       = 56
     chksum    = 0xb391
###[ DNS ]### 
        id        = 0
        qr        = 1L
        opcode    = QUERY
        aa        = 0L
        tc        = 0L
        rd        = 1L
        ra        = 1L
        z         = 0L
        ad        = 0L
        cd        = 0L
        rcode     = ok
        qdcount   = 1
        ancount   = 1
        nscount   = 0
        arcount   = 0
        \qd        \
         |###[ DNS Question Record ]### 
         |  qname     = 'www.google.com.'
         |  qtype     = A
         |  qclass    = IN
        \an        \
         |###[ DNS Resource Record ]### 
         |  rrname    = 'www.google.com.'
         |  type 

## Sniffing

In [7]:
def printPacket(p):
    destAddress = p[IP].dst
    sourceAddress = p[IP].src
    load = ''
    if Raw in p:
        load = p[Raw].load
    print('{} -> {}:{}'.format(sourceAddress, destAddress, load))
    
sniff(filter='tcp and port 21',prn=printPacket,count=10)

192.168.1.102 -> 213.138.116.78:PASV

213.138.116.78 -> 192.168.1.102:227 Entering Passive Mode (213,138,116,78,233,168).

192.168.1.102 -> 213.138.116.78:
192.168.1.102 -> 213.138.116.78:CWD /pub/FreeBSD/

213.138.116.78 -> 192.168.1.102:250 Directory successfully changed.

192.168.1.102 -> 213.138.116.78:LIST

213.138.116.78 -> 192.168.1.102:150 Here comes the directory listing.

192.168.1.102 -> 213.138.116.78:
213.138.116.78 -> 192.168.1.102:226 Directory send OK.

192.168.1.102 -> 213.138.116.78:


<Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>

## Writing packages to a pcap file

In [ ]:
def writePacket(p):
    wrpcap('scapy_example.pcap',p,append=True)

sniff(filter='tcp and port 21',prn=writePacket,count=10)